In [ ]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor

model_id = 'hongseongpil/Wav2Vec2.0_zeroth_Ko'
commit_id = 'b9d6b6a465dcad3fec4702483adeb1a76366f661'
model = Wav2Vec2ForCTC.from_pretrained(model_id, revision=commit_id,output_attentions=True)
processor = Wav2Vec2Processor.from_pretrained(model_id, revision=commit_id)

In [ ]:
from Tokenize_Kor import decompose_tokens
import torch
from pydub.effects import normalize
from pydub import AudioSegment 
from pydub.silence import detect_nonsilent
import numpy as np
def getAudioInput(source_path):
    try:
        audio = AudioSegment.from_file(source_path).set_frame_rate(16000).set_sample_width(2).set_channels(1)
        result = detect_nonsilent(audio,min_silence_len=200,silence_thresh=-60)
        newaudio = AudioSegment.empty()
        for index in result:
            newaudio += audio[index[0]:index[1]]
        newaudio = normalize(newaudio)
        return newaudio
    except Exception as e:
        print("오류 발생:", e)
def PrintAudioInfo(audio):
    channels = audio.channels
    sample_rate = audio.frame_rate
    print("Channels:", channels)
    print("Sample rate:", sample_rate)
    print("Duration: ", audio.duration_seconds)
    print("Bit depth:", audio.sample_width, "bits") 
    print("len samples:", len(np.array(audio.get_array_of_samples())))
    display(audio)

In [ ]:
audio_input = getAudioInput("녹음.wav")
model.eval()
input = processor(np.array(audio_input.get_array_of_samples(),dtype=np.float32), sampling_rate=16000, return_tensors="pt").input_values[0]
with torch.no_grad():
    input_values = torch.tensor(input).unsqueeze(0)
    logits = model(input_values).logits


In [ ]:
# k = 5  # Number of top candidates to consider
# topk_logits, topk_indices = torch.topk(logits, k, dim=-1)

# outputs = list()
# for i,v in enumerate(topk_indices[0]):
#     output= list()
#     for c in v:
#         output.append(processor.tokenizer.convert_ids_to_tokens(c.item()))
#     outputs.append(output)

# for i,v in enumerate(outputs):
#     print(topk_logits[0][i],v)

predlogits = torch.argmax(logits, dim=-1)[0]

In [ ]:
outputs = processor.decode(predlogits,output_char_offsets=True)
time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate

PrintAudioInfo(audio_input)
for w in outputs['char_offsets']:
    print('word : ',w['char'],'start_offset :',round (w[ "start_offset" ] * time_offset, 2 ),'end_offset :',round (w[ "end_offset" ] * time_offset, 2 ))